Question 1: Please help me find out the license thing

### first approach 

In [1]:
%time
# PDF to HTML
# http://macappstore.org/pdf2htmlex/
# https://github.com/coolwanglu/pdf2htmlEX/wiki/Command-Line-Options
import os
echo = "pdf2htmlEX --zoom 1.3"
for filename in os.listdir("../../../pdf_files"):
    if filename.endswith("pdf"):
        print(filename)
        #os.system(echo+" "+filename)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs
CMS_2014_0115_12304.pdf
CMS_2014_0115_58152.pdf
CMS_2014_0115_58434.pdf
CMS_2014_0115_56380.pdf
CMS_2014_0115_5633.pdf
CMS_2014_0115_52496.pdf
CMS_2014_0115_58186.pdf
CMS_2014_0115_52483.pdf
CMS_2014_0115_13187.pdf
CMS_2014_0115_12528.pdf
CMS_2014_0115_57882.pdf
CMS_2014_0115_9434.pdf
CMS_2014_0115_58168.pdf
CMS_2014_0115_13182.pdf
CMS_2014_0115_12505.pdf
CMS_2014_0115_58424.pdf
CMS_2014_0115_51400.pdf
CMS_2014_0115_58430.pdf
CMS_2014_0115_53363.pdf
CMS_2014_0115_55706.pdf
CMS_2014_0115_13035.pdf
CMS_2014_0115_58008.pdf
CMS_2014_0115_12895.pdf
CMS_2014_0115_13235.pdf
CMS_2014_0115_53188.pdf
CMS_2014_0115_58432.pdf
CMS_2014_0115_56218.pdf
CMS_2014_0115_54369.pdf
CMS_2014_0115_51921.pdf
CMS_2014_0115_54291.pdf
CMS_2014_0115_0059.pdf
CMS_2014_0115_13440.pdf
CMS_2014_0115_12762.pdf
CMS_2014_0115_56336.pdf
CMS_2014_0115_52959.pdf
CMS_2014_0115_56256.pdf
CMS_2014_0115_13244.pdf
CMS_2014_0115_53258.pdf
CMS_2014_0115_6167.pdf
CMS_

## second approach (current one)

In [2]:
%time
# PDF to HTML
# https://sourceforge.net/projects/pdftohtml/
# https://www.experts-exchange.com/videos/59892/Xpdf-PDFtoHTML-Command-Line-Utility-to-Convert-a-PDF-File-to-HTML.html
# https://www.xpdfreader.com/download.html

# make sure to have a folder named "pdf_files" with the pdfs
# make sure to have an empty folder named "html_files"
import os
echo = "pdftohtml"
for filename in os.listdir("../../../pdf_files"):
    if filename.endswith("pdf"):
        print(filename)
        file_pdf = "pdf_files/" + filename
        file_html = "html_files/" + filename
        #os.system(echo+" "+file_pdf+" "+file_html) 

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs
CMS_2014_0115_12304.pdf
CMS_2014_0115_58152.pdf
CMS_2014_0115_58434.pdf
CMS_2014_0115_56380.pdf
CMS_2014_0115_5633.pdf
CMS_2014_0115_52496.pdf
CMS_2014_0115_58186.pdf
CMS_2014_0115_52483.pdf
CMS_2014_0115_13187.pdf
CMS_2014_0115_12528.pdf
CMS_2014_0115_57882.pdf
CMS_2014_0115_9434.pdf
CMS_2014_0115_58168.pdf
CMS_2014_0115_13182.pdf
CMS_2014_0115_12505.pdf
CMS_2014_0115_58424.pdf
CMS_2014_0115_51400.pdf
CMS_2014_0115_58430.pdf
CMS_2014_0115_53363.pdf
CMS_2014_0115_55706.pdf
CMS_2014_0115_13035.pdf
CMS_2014_0115_58008.pdf
CMS_2014_0115_12895.pdf
CMS_2014_0115_13235.pdf
CMS_2014_0115_53188.pdf
CMS_2014_0115_58432.pdf
CMS_2014_0115_56218.pdf
CMS_2014_0115_54369.pdf
CMS_2014_0115_51921.pdf
CMS_2014_0115_54291.pdf
CMS_2014_0115_0059.pdf
CMS_2014_0115_13440.pdf
CMS_2014_0115_12762.pdf
CMS_2014_0115_56336.pdf
CMS_2014_0115_52959.pdf
CMS_2014_0115_56256.pdf
CMS_2014_0115_13244.pdf
CMS_2014_0115_53258.pdf
CMS_2014_0115_6167.pdf


In [3]:
import codecs
from bs4 import BeautifulSoup
import cssutils
import pandas as pd
import matplotlib.pyplot as plt
import os

In [4]:
def style_dict(style_tem):
    style_tem = style_tem[:-1]
    style_dic = dict(item.split(":") for item in style_tem.split(";"))
    return style_dic

In [5]:
def css_style(soup):
    selectors = {}
    for styles in soup.select('style'):
        css = cssutils.parseString(styles.encode_contents())
        for rule in css:
            if rule.type == rule.STYLE_RULE:
                style = rule.selectorText
                selectors[style] = {}
                for item in rule.style:
                    propertyname = item.name
                    value = item.value
                    selectors[style][propertyname] = value

    dict_css = {key.strip('#'): item for key, item in selectors.items()}
    return dict_css

In [6]:
def extract_features(path):
    with open(path) as raw_resuls:
        results = BeautifulSoup(raw_resuls, 'lxml')
    
    dict_css = css_style(results)
    lines = []
    text = []
    for element in results.find_all("body"):
        for stat in element.find_all("div"):
            data={}
            # gets the info from <div>
            div_dic = stat.attrs
            data.update(div_dic)

            # gets info from <span>
            span_dic = stat.find('span').attrs
            data.update(span_dic)

            # gets the features from style in <div>
            style_ = span_dic.get('style')
            style_span = style_dict(style_)
            data.update(style_span)

            # gets features of the font from css 
            font = span_dic.get('id')
            temp = dict_css.get(font)
            data.update(temp)

            # gets the features from style in <div>
            style_ = div_dic.get('style')
            style_temp = style_dict(style_)
            data.update(style_temp)
            
            d = pd.DataFrame.from_dict(data)
            
            text.append(stat.text)
            #print(stat.text)
            lines.append(d)

    dataframe = pd.concat(lines, axis=0).reset_index(drop=1)
    dataframe["text"] = pd.Series( (v for v in text))
    
    return dataframe

### reads all html files

In [7]:
%time
features = []
for root, dirs, files in os.walk("../../../html_files"):
    for file in files:
        print(file)
        if file.endswith(".html"):
            if file != "index.html":
                try:
                    file_path =  os.path.join(root, file)
                    print(file_path+ " found")
                    df = extract_features(file_path)
                    df = df.assign(page=file)
                    df = df.assign(document=root)
                    features.append(df)
                except:
                    print(file_path+" NOT found")

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.68 µs
page1.html
../../../html_files/CMS_2014_0115_12304.pdf/page1.html found
index.html
page8.png
page7.html
../../../html_files/CMS_2014_0115_12304.pdf/page7.html found
page6.html
../../../html_files/CMS_2014_0115_12304.pdf/page6.html found
page5.html
../../../html_files/CMS_2014_0115_12304.pdf/page5.html found
page4.html
../../../html_files/CMS_2014_0115_12304.pdf/page4.html found
page8.html
../../../html_files/CMS_2014_0115_12304.pdf/page8.html found
page6.png
page3.html
../../../html_files/CMS_2014_0115_12304.pdf/page3.html found
page7.png
page5.png
page4.png
page1.png
page3.png
page2.html
../../../html_files/CMS_2014_0115_12304.pdf/page2.html found
page2.png
page18.html
../../../html_files/CMS_2014_0115_58152.pdf/page18.html found
page1.html
../../../html_files/CMS_2014_0115_58152.pdf/page1.html found
page14.html
../../../html_files/CMS_2014_0115_58152.pdf/page14.html found
page9.png
index.html
page15.html
../../../html_fi

/Users/pranjalbajaj/anaconda3/envs/kpmgcapstone/lib/python3.7/site-packages/ipykernel_launcher.py:40: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



index.html
page3.html
../../../html_files/CMS_2014_0115_5633.pdf/page3.html found
page1.png
page3.png
page2.html
../../../html_files/CMS_2014_0115_5633.pdf/page2.html found
page2.png
page1.html
../../../html_files/CMS_2014_0115_52496.pdf/page1.html found
page9.png
index.html
page8.png
page7.html
../../../html_files/CMS_2014_0115_52496.pdf/page7.html found
page6.html
../../../html_files/CMS_2014_0115_52496.pdf/page6.html found
page9.html
../../../html_files/CMS_2014_0115_52496.pdf/page9.html found
page10.html
../../../html_files/CMS_2014_0115_52496.pdf/page10.html found
page5.html
../../../html_files/CMS_2014_0115_52496.pdf/page5.html found
page4.html
../../../html_files/CMS_2014_0115_52496.pdf/page4.html found
page11.html
../../../html_files/CMS_2014_0115_52496.pdf/page11.html found
page8.html
../../../html_files/CMS_2014_0115_52496.pdf/page8.html found
page6.png
page3.html
../../../html_files/CMS_2014_0115_52496.pdf/page3.html found
page7.png
page5.png
page4.png
page1.png
page3.png
pa

page10.png
page2.png
page1.html
../../../html_files/CMS_2014_0115_12505.pdf/page1.html found
index.html
page4.html
../../../html_files/CMS_2014_0115_12505.pdf/page4.html found
page3.html
../../../html_files/CMS_2014_0115_12505.pdf/page3.html found
page4.png
page1.png
page3.png
page2.html
../../../html_files/CMS_2014_0115_12505.pdf/page2.html found
page2.png
page1.html
../../../html_files/CMS_2014_0115_58424.pdf/page1.html found
index.html
page6.html
../../../html_files/CMS_2014_0115_58424.pdf/page6.html found
page5.html
../../../html_files/CMS_2014_0115_58424.pdf/page5.html found
page4.html
../../../html_files/CMS_2014_0115_58424.pdf/page4.html found
page6.png
page3.html
../../../html_files/CMS_2014_0115_58424.pdf/page3.html found
page5.png
page4.png
page1.png
page3.png
page2.html
../../../html_files/CMS_2014_0115_58424.pdf/page2.html found
page2.png
page1.html
../../../html_files/CMS_2014_0115_51400.pdf/page1.html found
page14.html
../../../html_files/CMS_2014_0115_51400.pdf/page14.ht

page6.html
../../../html_files/CMS_2014_0115_53188.pdf/page6.html found
page13.html
../../../html_files/CMS_2014_0115_53188.pdf/page13.html found
page9.html
../../../html_files/CMS_2014_0115_53188.pdf/page9.html found
page10.html
../../../html_files/CMS_2014_0115_53188.pdf/page10.html found
page5.html
../../../html_files/CMS_2014_0115_53188.pdf/page5.html found
page4.html
../../../html_files/CMS_2014_0115_53188.pdf/page4.html found
page11.html
../../../html_files/CMS_2014_0115_53188.pdf/page11.html found
page8.html
../../../html_files/CMS_2014_0115_53188.pdf/page8.html found
page6.png
page14.png
page3.html
../../../html_files/CMS_2014_0115_53188.pdf/page3.html found
page15.png
page7.png
page5.png
page17.png
page16.html
../../../html_files/CMS_2014_0115_53188.pdf/page16.html found
page16.png
page4.png
page12.png
page17.html
../../../html_files/CMS_2014_0115_53188.pdf/page17.html found
page13.png
page1.png
page3.png
page11.png
page2.html
../../../html_files/CMS_2014_0115_53188.pdf/page2.

...

page9.png
index.html
page15.html
../../../html_files/CMS_2014_0115_55350.pdf/page15.html found
page8.png
page12.html
../../../html_files/CMS_2014_0115_55350.pdf/page12.html found
page7.html
../../../html_files/CMS_2014_0115_55350.pdf/page7.html found
page6.html
../../../html_files/CMS_2014_0115_55350.pdf/page6.html found
page13.html
../../../html_files/CMS_2014_0115_55350.pdf/page13.html found
page9.html
../../../html_files/CMS_2014_0115_55350.pdf/page9.html found
page10.html
../../../html_files/CMS_2014_0115_55350.pdf/page10.html found
page5.html
../../../html_files/CMS_2014_0115_55350.pdf/page5.html found
page4.html
../../../html_files/CMS_2014_0115_55350.pdf/page4.html found
page11.html
../../../html_files/CMS_2014_0115_55350.pdf/page11.html found
page8.html
../../../html_files/CMS_2014_0115_55350.pdf/page8.html found
page6.png
page14.png
page3.html
../../../html_files/CMS_2014_0115_55350.pdf/page3.html found
page15.png
page7.png
page5.png
page4.png
page12.png
page13.png
page1.png
p

page13.png
page1.png
page3.png
page11.png
page2.html
../../../html_files/CMS_2014_0115_58260.pdf/page2.html found
page10.png
page2.png
page21.png
page18.html
../../../html_files/CMS_2014_0115_53135.pdf/page18.html found
page1.html
../../../html_files/CMS_2014_0115_53135.pdf/page1.html found
page20.png
page14.html
../../../html_files/CMS_2014_0115_53135.pdf/page14.html found
page9.png
index.html
page15.html
../../../html_files/CMS_2014_0115_53135.pdf/page15.html found
page8.png
page18.png
page19.html
../../../html_files/CMS_2014_0115_53135.pdf/page19.html found
page19.png
page12.html
../../../html_files/CMS_2014_0115_53135.pdf/page12.html found
page7.html
../../../html_files/CMS_2014_0115_53135.pdf/page7.html found
page6.html
../../../html_files/CMS_2014_0115_53135.pdf/page6.html found
page13.html
../../../html_files/CMS_2014_0115_53135.pdf/page13.html found
page9.html
../../../html_files/CMS_2014_0115_53135.pdf/page9.html found
page10.html
../../../html_files/CMS_2014_0115_53135.pdf/pa

index.html
page7.html
../../../html_files/CMS_2014_0115_12912.pdf/page7.html found
page6.html
../../../html_files/CMS_2014_0115_12912.pdf/page6.html found
page5.html
../../../html_files/CMS_2014_0115_12912.pdf/page5.html found
page4.html
../../../html_files/CMS_2014_0115_12912.pdf/page4.html found
page6.png
page3.html
../../../html_files/CMS_2014_0115_12912.pdf/page3.html found
page7.png
page5.png
page4.png
page1.png
page3.png
page2.html
../../../html_files/CMS_2014_0115_12912.pdf/page2.html found
page2.png
page1.html
../../../html_files/CMS_2014_0115_52488.pdf/page1.html found
page9.png
index.html
page8.png
page12.html
../../../html_files/CMS_2014_0115_52488.pdf/page12.html found
page7.html
../../../html_files/CMS_2014_0115_52488.pdf/page7.html found
page6.html
../../../html_files/CMS_2014_0115_52488.pdf/page6.html found
page9.html
../../../html_files/CMS_2014_0115_52488.pdf/page9.html found
page10.html
../../../html_files/CMS_2014_0115_52488.pdf/page10.html found
page5.html
../../../

page11.html
../../../html_files/CMS_2014_0115_58004.pdf/page11.html found
page8.html
../../../html_files/CMS_2014_0115_58004.pdf/page8.html found
page6.png
page14.png
page3.html
../../../html_files/CMS_2014_0115_58004.pdf/page3.html found
page7.png
page5.png
page4.png
page12.png
page13.png
page1.png
page3.png
page11.png
page2.html
../../../html_files/CMS_2014_0115_58004.pdf/page2.html found
page10.png
page2.png
page1.html
../../../html_files/CMS_2014_0115_52505.pdf/page1.html found
page9.png
index.html
page8.png
page12.html
../../../html_files/CMS_2014_0115_52505.pdf/page12.html found
page7.html
../../../html_files/CMS_2014_0115_52505.pdf/page7.html found
page6.html
../../../html_files/CMS_2014_0115_52505.pdf/page6.html found
page9.html
../../../html_files/CMS_2014_0115_52505.pdf/page9.html found
page10.html
../../../html_files/CMS_2014_0115_52505.pdf/page10.html found
page5.html
../../../html_files/CMS_2014_0115_52505.pdf/page5.html found
page4.html
../../../html_files/CMS_2014_0115_5

page4.html
../../../html_files/CMS_2014_0115_56370.pdf/page4.html found
page6.png
page3.html
../../../html_files/CMS_2014_0115_56370.pdf/page3.html found
page5.png
page4.png
page1.png
page3.png
page2.html
../../../html_files/CMS_2014_0115_56370.pdf/page2.html found
page2.png
page1.html
../../../html_files/CMS_2014_0115_52857.pdf/page1.html found
index.html
page6.html
../../../html_files/CMS_2014_0115_52857.pdf/page6.html found
page5.html
../../../html_files/CMS_2014_0115_52857.pdf/page5.html found
page4.html
../../../html_files/CMS_2014_0115_52857.pdf/page4.html found
page6.png
page3.html
../../../html_files/CMS_2014_0115_52857.pdf/page3.html found
page5.png
page4.png
page1.png
page3.png
page2.html
../../../html_files/CMS_2014_0115_52857.pdf/page2.html found
page2.png
page1.html
../../../html_files/CMS_2014_0115_58162.pdf/page1.html found
index.html
page4.html
../../../html_files/CMS_2014_0115_58162.pdf/page4.html found
page3.html
../../../html_files/CMS_2014_0115_58162.pdf/page3.html 

In [8]:
a = pd.concat(features, axis=0, sort=False).reset_index(drop=1)
a["Num_Words"] = a["text"].apply(lambda x: len(x.split()))
a["String_Length"] = a["text"].apply(lambda x: len(x))

In [9]:
len(a.groupby(['document']).size())

119

## Adding text features to the table

In [10]:
#a["First_Word"] = a["text"].apply(lambda x: x.split()[0].strip()) # Extract the first word from the string
#a["Last_Word"] = a['text'].apply(lambda x: x.split()[-1]) # Extract the last word from the string

a['Leading_Char_Upper'] = a['text'].str[0].str.isupper()    # Check is starting with uppercase character and assign boolen value
a['Leading_Char_Upper'] = a['Leading_Char_Upper'].map({True: '1', False: '0'})


a['Leading_Numeral'] = a['text'].str[0].str.isdigit() # Check is starting with numeric and assign boolen value 
a['Leading_Numeral'] = a['Leading_Numeral'].map({True: '1', False: '0'})

## Need to add roman numbers too


a['Ends_In_Period'] = a['text'].str.endswith(".") # Check ending with period and assign boolen value 
a['Ends_In_Period'] = a['Ends_In_Period'].map({True: '1', False: '0'})


a['Leading_Number_Period'] = a['text'].str.match("\d+\..*") # Check start with any nuerical combinations followed by ".", ex 1.
a['Leading_Number_Period'] = a['Leading_Number_Period'].map({True: '1', False: '0'})


a['Leading_Char_Period'] = a['text'].str.contains('^[A-Z,a-z]\.+') # Check start with any uppercase or lowercase character  followed by "." , ex A.
a['Leading_Char_Period'] = a['text'].map({True: '1', False: '0'})

a['Roman_Period'] = a['text'].str.contains('[V|I|X]\.+') ## Any roman numaral  followed by "."
a['Roman_Period'] = a['Roman_Period'].map({True: '1', False: '0'})



In [11]:
a.head()

,class,style,id,font-size,vertical-align,color,font-family,font-weight,font-style,position,...,document,line-height,Num_Words,String_Length,Leading_Char_Upper,Leading_Numeral,Ends_In_Period,Leading_Number_Period,Leading_Char_Period,Roman_Period
0,txt,font-size:8px;vertical-align:baseline;color:rg...,f6,8px,baseline,"rgba(0,0,0,1)",sans-serif,bold,italic,absolute,...,../../../html_files/CMS_2014_0115_12304.pdf,NaN,4,23,0,1,1,0,NaN,0
1,txt,font-size:8px;vertical-align:baseline;color:rg...,f6,8px,baseline,"rgba(0,0,0,1)",sans-serif,bold,italic,absolute,...,../../../html_files/CMS_2014_0115_12304.pdf,NaN,3,20,1,0,0,0,NaN,0
2,txt,font-size:12px;vertical-align:baseline;color:r...,f13,12px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,...,../../../html_files/CMS_2014_0115_12304.pdf,NaN,3,16,1,0,0,0,NaN,0
3,txt,font-size:12px;vertical-align:baseline;color:r...,f13,12px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,...,../../../html_files/CMS_2014_0115_12304.pdf,NaN,6,40,1,0,0,0,NaN,0
4,txt,font-size:12px;vertical-align:baseline;color:r...,f13,12px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,...,../../../html_files/CMS_2014_0115_12304.pdf,NaN,6,39,1,0,0,0,NaN,0


In [12]:
from webcolors import *
import numpy as np

In [22]:
len(a)

41910

In [23]:
%time
a['color_val'] = np.nan
for x in range(len(a)):
    try:
        a['color_val'][x]  = rgb_to_name(eval(a['color'][x].lstrip('rgba'))[:3])
        print(a['color_val'][x])
        
    except:
        pass
        

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 27.9 µs


/Users/pranjalbajaj/anaconda3/envs/kpmgcapstone/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blac

black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blac

black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blac

...

black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blac

black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blue
black
black
blue
black
blue
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blue
black
blue
black
blue
black
black
black
black
black
black
blue
black
black
blue
black
black
black
black
blue
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
b

black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blac

black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blue
black
blue
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blue
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blue
blue
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
bla

black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blac

In [24]:
a['color_val'].unique()

array(['black', nan, 'blue', 'white', 'grey', 'purple'], dtype=object)

In [25]:
len(a['color_val'])

41910

In [26]:
a['color_val'].value_counts()

black     38846
blue        205
white        30
grey         19
purple        1
Name: color_val, dtype: int64

In [ ]:
a['color'].apply(lambda x: rgb_to_name(eval(x.lstrip('rgba'))[:3]))

In [ ]:
a.groupby(['document']).size().sum()

In [ ]:
len(a)

In [ ]:
a.to_csv("documents.csv")

In [ ]:
a.shape

In [ ]:
a['font-weight'].hist()